In [1]:
import os
import math
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Conv3D, MaxPooling3D, LSTM
from tensorflow.keras.utils import to_categorical
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.linalg import pinv
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saniuzzamanrobin/ravdess-speech-all-audio-and-video")

print("Path to dataset files:", path)

100%|██████████| 6.39G/6.39G [01:13<00:00, 93.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/saniuzzamanrobin/ravdess-speech-all-audio-and-video/versions/1


In [3]:
df='/root/.cache/kagglehub/datasets/saniuzzamanrobin/ravdess-speech-all-audio-and-video/versions/1'
vid_path = os.path.join(df, 'All Videos')
vid_files = os.listdir(vid_path)
print("Video Files:", vid_files)

Video Files: ['02-01-08-02-01-01-20.mp4', '02-01-02-01-02-01-01.mp4', '02-01-06-01-01-02-07.mp4', '02-01-06-02-01-01-20.mp4', '02-01-03-02-02-02-09.mp4', '02-01-03-01-01-02-02.mp4', '02-01-03-02-02-02-20.mp4', '02-01-07-02-02-01-12.mp4', '02-01-06-02-01-02-22.mp4', '02-01-05-02-02-02-23.mp4', '02-01-07-01-02-02-16.mp4', '02-01-04-02-01-02-12.mp4', '02-01-08-02-02-02-24.mp4', '02-01-06-01-02-01-21.mp4', '02-01-01-01-01-01-03.mp4', '02-01-06-01-02-02-05.mp4', '02-01-01-01-01-01-15.mp4', '02-01-08-02-01-02-08.mp4', '02-01-02-01-02-01-18.mp4', '02-01-05-01-02-02-15.mp4', '02-01-01-01-02-01-15.mp4', '02-01-08-01-01-01-10.mp4', '02-01-08-01-01-01-18.mp4', '02-01-06-01-02-02-11.mp4', '02-01-03-01-01-02-05.mp4', '02-01-08-01-01-02-03.mp4', '02-01-05-01-02-02-21.mp4', '02-01-04-02-01-02-22.mp4', '02-01-06-02-02-02-15.mp4', '02-01-08-02-01-02-21.mp4', '02-01-03-02-01-01-13.mp4', '02-01-02-02-01-01-19.mp4', '02-01-08-01-01-01-22.mp4', '02-01-08-01-01-02-16.mp4', '02-01-03-01-01-02-16.mp4', '02-01

In [4]:
# Define the emotions present in the dataset
emotions = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}
# Paths to audio and video directories
audio_dir = "/root/.cache/kagglehub/datasets/saniuzzamanrobin/ravdess-speech-all-audio-and-video/versions/1/All Audios"
video_dir = "/root/.cache/kagglehub/datasets/saniuzzamanrobin/ravdess-speech-all-audio-and-video/versions/1/All Videos"

In [ ]:
max_audio_frames = 180
audio_features = []
audio_labels = []

# Loop through the audio directory
for filename in os.listdir(audio_dir):
    if filename.endswith(".wav"):
        # Extract the emotion label from the filename
        emotion_label = emotions[filename.split("-")[2]]

        # Load the audio file
        audio_path = os.path.join(audio_dir, filename)
        # Load audio file using librosa
        audio, sr = librosa.load(audio_path)
        # Extract audio features using librosa
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20)
        # Append MFCC features and emotion label to the lists
        audio_features.append(mfcc.T)  # Transpose MFCC to have (num_frames, n_mfcc) shape
        audio_labels.append(emotion_label)

# Apply padding or trimming to make audio features consistent in length
for i in range(len(audio_features)):
    num_frames = audio_features[i].shape[0]
    if num_frames < max_audio_frames:
        # Padding: Add zeros to the end to match the desired length
        padding = np.zeros((max_audio_frames - num_frames, audio_features[i].shape[1]))
        audio_features[i] = np.vstack((audio_features[i], padding))
    elif num_frames > max_audio_frames:
        # Trimming: Remove extra frames to match the desired length
        audio_features[i] = audio_features[i][:max_audio_frames]

# Convert the audio_features list to a numpy array
audio_features = np.array(audio_features)

In [ ]:
print(audio_features.shape)


In [ ]:
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Add more audio features (e.g., mel-spectrogram, chroma)
def extract_audio_features(audio_path, sr=22050):
    audio, sr = librosa.load(audio_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)

    # Stack features together (you can experiment with different combinations)
    features = np.vstack([mfcc, mel_spectrogram, chroma])
    return features.T

# Update the model to include LSTM
audio_model = Sequential()

# Add convolutional layers
audio_model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(180, 33)))  # Adjust input shape for added features
audio_model.add(BatchNormalization())
audio_model.add(MaxPooling1D(pool_size=2))

audio_model.add(Conv1D(128, kernel_size=3, activation='relu'))
audio_model.add(BatchNormalization())
audio_model.add(MaxPooling1D(pool_size=2))

# Add LSTM to capture temporal dependencies
audio_model.add(LSTM(128, return_sequences=False))

# Fully connected layers
audio_model.add(Dense(128, activation='relu'))
audio_model.add(Dropout(0.5))  # Dropout for regularization
audio_model.add(Dense(64, activation='relu'))
audio_model.add(Dense(8, activation='softmax'))

# Compile the model
audio_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

audio_model.summary()

# Train the updated audio model
audio_model_history = audio_model.fit(audio_features_train, audio_labels_train, epochs=25, batch_size=32, validation_split=0.2)

# Evaluate the model
audio_loss, audio_accuracy = audio_model.evaluate(audio_features_test, audio_labels_test)
print("Audio Model Loss:", audio_loss)
print("Audio Model Accuracy:", audio_accuracy)


In [ ]:
def exclude_frames(video_frames, frame_rate, exclude_duration=0.4):
    exclude_frames_start = math.ceil(frame_rate * exclude_duration)
    exclude_frames_end = math.floor(frame_rate * exclude_duration)
    return video_frames[exclude_frames_start:-exclude_frames_end]

batch_size = 10
cnt = 0
video_features = []
video_labels = []

for i in range(0, len(os.listdir(video_dir)), batch_size):
    video_batch = os.listdir(video_dir)[i : i + batch_size]

    for filename in video_batch:
        if filename.endswith(".mp4"):
            # Extract the emotion label from the filename
            emotion_label = emotions[filename.split("-")[2]]

            # Load the video file
            video_path = os.path.join(video_dir, filename)
            cap = cv2.VideoCapture(video_path)

            # Calculate the frame rate of the video
            frame_rate = cap.get(cv2.CAP_PROP_FPS)
            frames = []
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame = frame[120:662, 400:912]
                frame = cv2.resize(frame, (80, 70))
                frames.append(frame)

            cap.release()

            # Exclude frames at the start and end of the video
            frames = exclude_frames(frames, frame_rate, exclude_duration=0.4)
            # Pad or truncate frames to have a consistent number of frames (20 frames)
            if len(frames) < 30:
                frames.extend([frames[-1]] * (30 - len(frames)))
            else:
                frames = frames[:30]

            if(cnt%100==0):
                print(cnt)
            cnt+=1
            video_features.append(frames)
            video_labels.append(emotion_label)

video_features = np.array(video_features)

In [ ]:
print(video_features.shape)


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import TimeDistributed, Conv3D, MaxPooling3D, LSTM, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
import numpy as np
import cv2

# Load pretrained VGG16 model for feature extraction (excluding the top classification layers)
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(70, 80, 3))

# Define video model
video_model = Sequential()

# Add VGG16 for feature extraction (applied frame-wise using TimeDistributed)
video_model.add(TimeDistributed(vgg16, input_shape=(30, 70, 80, 3)))  # 30 frames, each 70x80 resolution with 3 channels (RGB)
video_model.add(TimeDistributed(Flatten()))  # Flatten VGG16 output

# Add 3D CNN layers to capture temporal dependencies
video_model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
video_model.add(BatchNormalization())
video_model.add(MaxPooling3D(pool_size=(2, 2, 2)))

video_model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu'))
video_model.add(BatchNormalization())
video_model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Flatten and fully connected layers
video_model.add(Flatten())
video_model.add(Dense(128, activation='relu'))
video_model.add(Dropout(0.5))  # Dropout to prevent overfitting
video_model.add(Dense(64, activation='relu'))
video_model.add(Dense(8, activation='softmax'))  # 8 output classes (for each emotion)

# Compile the video model
video_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
video_model.summary()

# Train the updated video model
video_model_history = video_model.fit(video_features_train_reshaped, video_labels_train, batch_size=32, epochs=15, validation_split=0.2)

# Evaluate the model
video_loss, video_accuracy = video_model.evaluate(video_features_test_reshaped, video_labels_test)
print("Video Model Loss:", video_loss)
print("Video Model Accuracy:", video_accuracy)


In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, LSTM, Input, concatenate

# Get the input tensors of the video and audio models
# Instead of using audio_model.input/video_model.input,
# access the input layer by index 0
audio_input = audio_model.layers[0].input
video_input = video_model.layers[0].input

# Get the output tensors of the video and audio models
video_output = video_model.layers[-2].output
audio_output = audio_model.layers[-2].output

# Concatenate the video and audio outputs
combined = concatenate([video_output, audio_output])

# Add additional layers for fusion
fusion_layer = Dense(128, activation='relu')(combined)
fusion_layer = Dense(64, activation='relu')(fusion_layer)

# Final output layer
output_layer = Dense(8, activation='softmax')(fusion_layer)

# Create the fusion model
# Use audio_input and video_input defined earlier
fusion_model = Model(inputs=[audio_input, video_input], outputs=output_layer)

# Compile the fusion model
fusion_model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

# Print the summary of the fusion model
fusion_model.summary()
# Train fusion model
fusion_history = fusion_model.fit([audio_features_train, video_features_train_reshaped], audio_labels_train, batch_size=64, epochs=12, validation_split=0.2)

# Evaluate the fusion model
fusion_score = fusion_model.evaluate([audio_features_test, video_features_test_reshaped], audio_labels_test)
print("Fusion Model Testing - Loss: {:.4f}, Accuracy: {:.4f}".format(fusion_score[0], fusion_score[1]))

# Calculate accuracy score
fusion_y_pred = fusion_model.predict([audio_features_eval, video_features_eval_reshaped]).argmax(axis=-1)
audio_labels_eval_int = np.argmax(audio_labels_eval, axis=1)
fusion_accuracy = accuracy_score(audio_labels_eval_int, fusion_y_pred)
print("Fusion Model Accuracy: {:.4f}".format(fusion_accuracy))

In [ ]:
# Function to preprocess audio (same as in training)
def preprocess_audio(audio_path, max_audio_frames=180):
    audio, sr = librosa.load(audio_path)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20)

    # Padding or trimming to match the model's input shape
    if mfcc.shape[1] < max_audio_frames:
        padding = np.zeros((mfcc.shape[0], max_audio_frames - mfcc.shape[1]))
        mfcc = np.hstack((mfcc, padding))
    elif mfcc.shape[1] > max_audio_frames:
        mfcc = mfcc[:, :max_audio_frames]

    return mfcc.T  # Transpose to match input shape

# Function to preprocess video (same as in training)
def preprocess_video(video_path, frame_count=30):
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = frame[120:662, 400:912]
        frame = cv2.resize(frame, (80, 70))
        frames.append(frame)
    cap.release()

    # Exclude frames and pad or trim to 30 frames
    frames = exclude_frames(frames, frame_rate, exclude_duration=0.4)
    if len(frames) < frame_count:
        frames.extend([frames[-1]] * (frame_count - len(frames)))
    else:
        frames = frames[:frame_count]

    return np.array(frames)

# Load new audio and video files
new_audio_path = "/content/03-01-02-02-01-02-05.wav"
new_video_path = "/content/01-01-03-01-01-01-23.mp4"

# Preprocess the new audio and video
new_audio_features = preprocess_audio(new_audio_path)
new_video_features = preprocess_video(new_video_path)

# Reshape video features to match the model input shape
new_video_features = np.expand_dims(new_video_features, axis=-1)  # Add channel dimension
new_audio_features = np.expand_dims(new_audio_features, axis=0)   # Add batch dimension
new_video_features = np.expand_dims(new_video_features, axis=0)   # Add batch dimension

# Predict using the fusion model
fusion_prediction = fusion_model.predict([new_audio_features, new_video_features])

# Get the predicted emotion label
predicted_emotion_index = np.argmax(fusion_prediction, axis=1)
predicted_emotion = label_encoder.inverse_transform(predicted_emotion_index)

print("Predicted Emotion:", predicted_emotion)


In [ ]:
# Get predictions from the individual audio model
audio_prediction = audio_model.predict(new_audio_features)
audio_predicted_emotion_index = np.argmax(audio_prediction, axis=1)
audio_predicted_emotion = label_encoder.inverse_transform(audio_predicted_emotion_index)

# Get predictions from the individual video model
video_prediction = video_model.predict(new_video_features)
video_predicted_emotion_index = np.argmax(video_prediction, axis=1)
video_predicted_emotion = label_encoder.inverse_transform(video_predicted_emotion_index)

# Get predictions from the fusion model (already done in previous code)
fusion_prediction = fusion_model.predict([new_audio_features, new_video_features])
fusion_predicted_emotion_index = np.argmax(fusion_prediction, axis=1)
fusion_predicted_emotion = label_encoder.inverse_transform(fusion_predicted_emotion_index)

# Print the results
print("Audio Model Predicted Emotion:", audio_predicted_emotion[0])
print("Video Model Predicted Emotion:", video_predicted_emotion[0])
print("Fusion Model Predicted Emotion:", fusion_predicted_emotion[0])

# Compare which one matches the fusion prediction
if fusion_predicted_emotion[0] == audio_predicted_emotion[0] and fusion_predicted_emotion[0] != video_predicted_emotion[0]:
    print("The fusion prediction is more influenced by the audio.")
elif fusion_predicted_emotion[0] == video_predicted_emotion[0] and fusion_predicted_emotion[0] != audio_predicted_emotion[0]:
    print("The fusion prediction is more influenced by the video.")
else:
    print("Both audio and video contributed equally to the fusion prediction.")
